## Inference with the LLama Model

In this notebook, we will call the croissant LLM to generate the response from our questions. We will compare the first approach that call the model directly from the transformer library and then the second approach that hit the llam-cpp api.


In [4]:
from transformers import LlamaForCausalLM, AutoTokenizer, set_seed
import torch

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
set_seed(42)

In [6]:
model_name = "croissantllm/CroissantLLMChat-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto", offload_folder="offload")

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.79s/it]


In [7]:
from src.rag.components.retriever import HybridRetriever
model_id = "camembert-base"
spacy_model = 'fr_core_news_md'

In [8]:
retriever = HybridRetriever(model_id=model_id, spacy_model=spacy_model)

No sentence-transformers model found with name camembert-base. Creating a new one with mean pooling.
/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
questions = ["Quand l’Ordonnance présidentielle a-t-elle été lue sur le plateau de la Radiotélévision nationale congolaise (RTNC)?",
             "Qui a été nommé pour remplacer Emmanuel Ramazani Shadary au poste de vice-Premier ministre et ministre de l’Intérieur et sécurité?",
             "Où et quand Henri Mova Sakanyi est-il né?",
             "Quelle est la carrière politique de Henri Mova Sakanyi en République démocratique du Congo?",
             "Quel est le poste actuel de Henri Mova Sakanyi au sein du Parti du peuple pour la Reconstruction et la Démocratie (PPRD)?"]

In [ ]:
template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

In [ ]:
from jinja2 import Template

### Using The Transformer Model

In [15]:
def generate_chat_input(query:str, documents:list) -> str:
    prompt_template  = """
        Context:
        {% for document in documents %}
            {{ document }}
        {% endfor %}

        Question: {{question}}
        Answer:
        """
    template = Template(prompt_template)
    prompt = template.render(documents=documents, question=query)
    
    chat_input = [
        {"role": "system", "content": "Given the Context:, answer the question in french."},
      {"role": "user", "content": prompt},
    ]
    
    return chat_input

In [10]:
def parse_response(text):
    """ Split the text inside the  <|im_start|> assistant <|im_end|> tags and then split the new line text  and return the pair question and response"""

    # use regex to get the text inside the <|im_start|> assistant <|im_end|> tags
    text = text.split("<|im_start|> assistant")[1].split("<|im_end|>")[0]
    return text

In [7]:
def generate_answer(chat_input:str) -> str:
    """this function generates an answer to a question given a chat input

    Args:
        chat_input (str): _description_

    Returns:
        str: _description_
    """
   
    generation_args = {
        "max_new_tokens": 512,
        "do_sample": True,
        "temperature": 0.3,
        "top_p": 0.90,
        "top_k": 40,
        "repetition_penalty": 1.05,
        "eos_token_id": [tokenizer.eos_token_id, 32000],
    }


    inputs = tokenizer(chat_input, return_tensors="pt").to(model.device)
    tokens = model.generate(**inputs, **generation_args)
    input_ids = inputs["input_ids"]
    generated_token = tokens[0][input_ids.shape[-1]:]
    return tokenizer.decode(generated_token)

In [12]:
from unicodedata import normalize

In [13]:
questions[1]

'Qui a été nommé pour remplacer Emmanuel Ramazani Shadary au poste de vice-Premier ministre et ministre de l’Intérieur et sécurité?'

In [27]:
for question in questions[:1]:
    documents = retriever.run(question)
    chat_input = generate_chat_input(question, documents)
    chat_tokens = tokenizer.apply_chat_template(
        chat_input, tokenize=False, add_generation_prompt=True)
    answer = generate_answer(chat_tokens)
    answer = parse_response(answer)
    print("the answer to the question {} is: __ \n {}".format(question, answer))

    print(50 * "-")

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


the answer to the question Quand l’Ordonnance présidentielle a-t-elle été lue sur le plateau de la Radiotélévision nationale congolaise (RTNC)? is: __ 
 L'Ordonnance présidentielle a été lue hier mardi 20 février 2018 sur le plateau de la Radiotélévision nationale congolaise (RTNC), dans l'émission "Histoire du Congo". La réponse ne peut pas être donnée car elle dépend de l'ordre chronologique ou logique des événements décrits dans la question.<|im_end|>
--------------------------------------------------


In [28]:
chat_tokens

'<|im_start|>system\nGiven the Context:, answer the question in french.<|im_end|>\n<|im_start|>user\n\n        Context:\n        \n            (KINSHASA)- Le présentateur de la très controversée émission ‘‘Histoire du Congo’’ diffusée sur la télévision publique-Rtnc, Lushima Ndjate continue à prester normalement comme si le conseil supérieur de l’audiovisuel et de la communication n’avait pris aucune décision à son encontre.\n        \n            L’Ordonnance présidentielle a été lue hier, mardi 20 février 2018 sur le plateau de la Radiotélévision nationale congolaise (RTNC) dans l’édition de 20 heures. Henri Mova Sakanyi remplace ainsi Emmanuel Ramazani Shadary, vice-Premier ministre et ministre de l’Intérieur et sécurité. Mova\n        \n            Adolphe Lumanu Mulenda Bwana N’Sefu a fait montre de capacité et de crédibilité de son organisme, l’Office national d’identification de la population créée par ordonnance présidentielle dont les membres venaient de l’être par la même voi

#### Hit the LLama API

Bellow is the code that call the llama server Api to get the message.

In [28]:

# Specify the URL
API_URL = 'http://142.132.172.241:8000/completion'

In [9]:
questions[1]

'Qui a été nommé pour remplacer Emmanuel Ramazani Shadary au poste de vice-Premier ministre et ministre de l’Intérieur et sécurité?'

In [10]:
documents = retriever.run(questions[1])


In [12]:
for document in documents:
    print(document)
    print(50 * "-")

officiers supérieurs de la PNC, des FARDC et des responsables des services de sécurité. Il faut noter également la présence des membres de la Commission Défense et Sécurité de l’Assemblée nationale. Selon le Vice-Premier Ministre en charge de l’Interieur et Sécurité qui a fait le point
--------------------------------------------------
Il a désigné son dauphin en août 2018.
Il s'agit de son ancien ministre de l'intérieur Emmanuel Ramazani Shadary.
--------------------------------------------------
Par Joe Kazumba/stagiaireRécemment secrétaire général du Parti au pouvoir, PPRD, Henry Mova Sakanyi vient d’être nommé vice-premier ministre en charge de l’intérieur par ordonnance présidentielle, mardi 20 février. Il remplace Emmanuel Shadary à ce poste.Mova Sakanyi fut ministre des
--------------------------------------------------
Le décor est déjà planté. Les invités sont bien présents. Dans quelques heures le vice-premier ministre, ministre de l’intérieur et sécurité, Ramazani Shadary va

In [18]:
chat_input = generate_chat_input(questions[1], documents)

In [19]:
chat_input

[{'role': 'system',
  'content': 'Given the Context:, answer the question in french.'},
 {'role': 'user',
  'content': "\n        Context:\n        \n            officiers supérieurs de la PNC, des FARDC et des responsables des services de sécurité. Il faut noter également la présence des membres de la Commission Défense et Sécurité de l’Assemblée nationale. Selon le Vice-Premier Ministre en charge de l’Interieur et Sécurité qui a fait le point\n        \n            Il a désigné son dauphin en août 2018.\nIl s'agit de son ancien ministre de l'intérieur Emmanuel Ramazani Shadary.\n        \n            Par Joe Kazumba/stagiaireRécemment secrétaire général du Parti au pouvoir, PPRD, Henry Mova Sakanyi vient d’être nommé vice-premier ministre en charge de l’intérieur par ordonnance présidentielle, mardi 20 février. Il remplace Emmanuel Shadary à ce poste.Mova Sakanyi fut ministre des\n        \n            Le décor est déjà planté. Les invités sont bien présents. Dans quelques heures le 

In [20]:
chat_tokens = tokenizer.apply_chat_template(
    chat_input, tokenize=False, add_generation_prompt=True)

In [21]:
import requests
import json

In [22]:
tokenizer.eos_token

'<|im_end|>'

In [29]:
def generate_response_from_llama_api(prompt:str) ->str:
    """ 
    This function sends a post request to the llama api and returns the response.
    """
    headers = {
        'Content-Type': 'application/json',
    }


    data = {
        "prompt": prompt,
        "n_predict": 128,
        "temperature": 0.3,
        "top_k": 40,
        "top_p": 0.90,
        "stopped_eos": True,
        "repeat_penalty": 1.05,
        "stop": ["assistant", tokenizer.eos_token],
        "seed": 42
    }


    json_data = json.dumps(data)

    # Send the POST request
    try:
        response = requests.post(API_URL, headers=headers, data=json_data)
        return response.json()["content"]
    except Exception as e:
        print(e)
        return None

In [24]:
answer = generate_response_from_llama_api(chat_tokens)

In [25]:
answer

'           \nRéponse: Henry Moova Sakanyi.'

In [30]:
for question in questions:
    documents = retriever.run(question)
    chat_input = generate_chat_input(question, documents)
    prompt = tokenizer.apply_chat_template(
        chat_input, tokenize=False, add_generation_prompt=True)
    answer = generate_response_from_llama_api(prompt)
    print("the answer to the question {} is: __ \n {}".format(question, answer))

    print(50 * "-")

the answer to the question Quand l’Ordonnance présidentielle a-t-elle été lue sur le plateau de la Radiotélévision nationale congolaise (RTNC)? is: __ 
  La Présidentielle a été lue sur le plateau de la Radio-Télévision Nationale Congolaise (RTNC) dans l'édition du journal de 20 heures par la porte-parole du Chef de l'État, Tiina Shadary.
--------------------------------------------------
the answer to the question Qui a été nommé pour remplacer Emmanuel Ramazani Shadary au poste de vice-Premier ministre et ministre de l’Intérieur et sécurité? is: __ 
            
Réponse:
Le Vice-Premier Ministre en charge de l'Intérieur et Sécurité a été désigné pour remplacer Emmanuel Ramazaani Shadary au poste de vice-Premier ministre et ministre de l’Intérieur et sécurité.
--------------------------------------------------
the answer to the question Où et quand Henri Mova Sakanyi est-il né? is: __ 
  Henri Mova Sakanyi est né le 15 février 2018.
--------------------------------------------------
t

### Using the Class to generate the response

In [2]:
from src.rag.components.generator import LLamaCppGeneratorComponent
from src.rag.components.retriever import HybridRetriever

In [5]:
hybrid_retriever = HybridRetriever(model_id=model_id, spacy_model=spacy_model)

No sentence-transformers model found with name camembert-base. Creating a new one with mean pooling.
/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
sample_question = questions[4]

In [8]:
documents = hybrid_retriever.run(sample_question)

In [12]:
# Specify the URL
API_URL = 'http://localhost:8001'

In [13]:
generator_component = LLamaCppGeneratorComponent(api_url=API_URL)

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
assert generator_component._ping_api()

In [20]:
answer = generator_component.run(sample_question, documents)

In [21]:
print("the answer to the question {} is: __ \n {}".format(sample_question, answer))

the answer to the question Quel est le poste actuel de Henri Mova Sakanyi au sein du Parti du peuple pour la Reconstruction et la Démocratie (PPRD)? is: __ 
  Henri Mova Sakanyi est actuellement secrétaire exécutive de la direction politique du PPRD.


In [31]:
questions

['Quand l’Ordonnance présidentielle a-t-elle été lue sur le plateau de la Radiotélévision nationale congolaise (RTNC)?',
 'Qui a été nommé pour remplacer Emmanuel Ramazani Shadary au poste de vice-Premier ministre et ministre de l’Intérieur et sécurité?',
 'Où et quand Henri Mova Sakanyi est-il né?',
 'Quelle est la carrière politique de Henri Mova Sakanyi en République démocratique du Congo?',
 'Quel est le poste actuel de Henri Mova Sakanyi au sein du Parti du peuple pour la Reconstruction et la Démocratie (PPRD)?']

### How the generation works in practice

In [1]:
a =2

- # Let us start with a simple question: 


In [10]:
questions[1]

'Qui a été nommé pour remplacer Emmanuel Ramazani Shadary au poste de vice-Premier ministre et ministre de l’Intérieur et sécurité?'

In [33]:
keywords = retriever.perform_keyword_extraction(questions[1])

Now let peroform the keyword search to find out what will be genreated.

In [35]:
keywords_search_results = retriever.keyword_search(keywords)

With the keywords search results, let us perform, embeddings search